In [1]:
import tensorflow as tf
import math

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True, reshape=False, validation_size=0)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
x=mnist.train.images
y=mnist.train.labels

In [4]:
mnist.test.images.shape

(10000, 28, 28, 1)

In [5]:
print (x.shape)
print (y.shape)

(60000, 28, 28, 1)
(60000, 10)


In [6]:
X = tf.placeholder("float", [None,28, 28, 1])
Y_=tf.placeholder("float",[None,10])

In [7]:
# variable learning rate
lr = tf.placeholder(tf.float32)
# Probability of keeping a node during dropout = 1.0 at test time (no dropout) and 0.75 at training time
pkeep = tf.placeholder(tf.float32)

In [8]:
# three convolutional layers with their channel counts, and a
# fully connected layer (tha last layer has 10 softmax neurons)
K = 6  # first convolutional layer output depth
L = 12  # second convolutional layer output depth
M = 24  # third convolutional layer
N = 200  # fully connected layer

In [9]:
W1 = tf.Variable(tf.truncated_normal([5, 5, 1, K], stddev=0.1))  # 5x5 patch, 1 input channel, K output channels
B1 = tf.Variable(tf.ones([K])/10)

W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1))
B2 = tf.Variable(tf.ones([L])/10)

W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1))
B3 = tf.Variable(tf.ones([M])/10)

W4 = tf.Variable(tf.truncated_normal([7 * 7 * M, N], stddev=0.1))
B4 = tf.Variable(tf.ones([N])/10)

W5 = tf.Variable(tf.truncated_normal([N, 10], stddev=0.1))
B5 = tf.Variable(tf.ones([10])/10)

In [10]:
# The model
stride = 1  # output is 28x28
Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
stride = 2  # output is 14x14
Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
stride = 2  # output is 7x7
Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

In [11]:
# reshape the output from the third convolution for the fully connected layer
YY = tf.reshape(Y3, shape=[-1, 7 * 7 * M])

In [12]:
Y4 = tf.nn.relu(tf.matmul(YY, W4) + B4)
YY4 = tf.nn.dropout(Y4, pkeep)

Ylogits = tf.matmul(YY4, W5) + B5
Y = tf.nn.softmax(Ylogits)

In [13]:
# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
# problems with log(0) which is NaN
cost = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)
cost = tf.reduce_mean(cost)

In [14]:
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
# training step, the learning rate is a placeholder
optimizer = tf.train.AdamOptimizer(lr).minimize(cost)

In [16]:
# learning rate decay
max_learning_rate = 0.003
min_learning_rate = 0.0001
decay_speed = 2000.0 # 0.003-0.0001-2000=>0.9826 done in 5000 iterations

In [17]:
training_epochs = 25
batch_size = 100

In [18]:
init =tf.global_variables_initializer()

In [19]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        c=0.0
        avg_cost=0.0
        # Loop over all batches
        for i in range(total_batch):
            
            learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-i/decay_speed)
            
            batch_X, batch_Y = mnist.train.next_batch(batch_size)
            #print(batch_X.shape)
            _,c=sess.run([optimizer,cost], feed_dict={X:batch_X,Y_:batch_Y,lr: learning_rate,pkeep: 0.75})
            avg_cost += c / total_batch

        print("epchos {} cross entropy={}".format(epoch+1,avg_cost))
            
    print ("Optimization Finished!")

    # Test model
    #correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy for 3000 examples
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print( "Accuracy:", accuracy.eval({X: mnist.test.images, Y_: mnist.test.labels, pkeep: 1.0}))

epchos 1 cross entropy=0.21273899755828712
epchos 2 cross entropy=0.06333656027835477
epchos 3 cross entropy=0.04502295880229211
epchos 4 cross entropy=0.03791754350822881
epchos 5 cross entropy=0.030956107688083025
epchos 6 cross entropy=0.02833300012425755
epchos 7 cross entropy=0.02511570712951652
epchos 8 cross entropy=0.021002320980987848
epchos 9 cross entropy=0.02087026205588093
epchos 10 cross entropy=0.017188650018027433
epchos 11 cross entropy=0.017637365239546253
epchos 12 cross entropy=0.017380545436966117
epchos 13 cross entropy=0.015344708636331856
epchos 14 cross entropy=0.017440930747889084
epchos 15 cross entropy=0.013025540395898722
epchos 16 cross entropy=0.01600772949344597
epchos 17 cross entropy=0.012758348868407364
epchos 18 cross entropy=0.011427069986670898
epchos 19 cross entropy=0.012528469456061286
epchos 20 cross entropy=0.01527274708223404
epchos 21 cross entropy=0.010444954463907457
epchos 22 cross entropy=0.010583849752169389
epchos 23 cross entropy=0.01